# Tutorial 4: solving optimal power flow with MadNLP

With the previous tutorial, we have now all the elements to implement
the optimal power flow problem with MadNLP. It remains to add to the model
the following elements:

- an objective penalizing the cost to run each power generator.
- line flow constraints;
- voltage angle constraints;

We start the tutorial again by importing the usual packages:

In [ ]:
using LinearAlgebra
using SparseArrays

using NLPModels
using ExaModels

using JLD2

include("utils.jl")

We import a small instance:

In [ ]:
data = JLD2.load("instances/case9.jld2")["data"]
ngen = length(data.gen)
nbus = length(data.bus)
nlines = length(data.branch)

## Optimal power flow model

We implement the AC optimal power flow problem using ExaModels in the
following function `acopf_model`:

In [ ]:
function acopf_model(
    data;
    backend = nothing,
    T = Float64,
    kwargs...,
)
    w = ExaModels.ExaCore(T; backend = backend)
    va = ExaModels.variable(w, length(data.bus))
    vm = ExaModels.variable(
        w,
        length(data.bus);
        start = data.vm0,
        lvar = data.vmin,
        uvar = data.vmax,
    )
    pg = ExaModels.variable(w, length(data.gen); start=data.pg0, lvar = data.pmin, uvar = data.pmax)
    qg = ExaModels.variable(w, length(data.gen); start=data.qg0, lvar = data.qmin, uvar = data.qmax)
    p = ExaModels.variable(w, length(data.arc); lvar = -data.rate_a, uvar = data.rate_a)
    q = ExaModels.variable(w, length(data.arc); lvar = -data.rate_a, uvar = data.rate_a)

    o = ExaModels.objective(
        w,
        g.cost1 * pg[g.i]^2 + g.cost2 * pg[g.i] + g.cost3 for g in data.gen
    )

    c1 = ExaModels.constraint(w, va[i] for i in data.ref_buses)

Active power flow, FR

In [ ]:
    c2 = ExaModels.constraint(
        w,
        p[b.f_idx] - b.c5 * vm[b.f_bus]^2 -
        b.c3 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
        b.c4 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
        b in data.branch
    )

Reactive power flow, FR

In [ ]:
    c3 = ExaModels.constraint(
        w,
        q[b.f_idx] +
        b.c6 * vm[b.f_bus]^2 +
        b.c4 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
        b.c3 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
        b in data.branch
    )

Active power flow, TO

In [ ]:
    c4 = ExaModels.constraint(
        w,
        p[b.t_idx] - b.c7 * vm[b.t_bus]^2 -
        b.c1 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
        b.c2 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
        b in data.branch
    )

Reactive power flow, TO

In [ ]:
    c5 = ExaModels.constraint(
        w,
        q[b.t_idx] +
        b.c8 * vm[b.t_bus]^2 +
        b.c2 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
        b.c1 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
        b in data.branch
    )

Voltage angle difference

In [ ]:
    c6 = ExaModels.constraint(
        w,
        va[b.f_bus] - va[b.t_bus] for b in data.branch;
        lcon = data.angmin,
        ucon = data.angmax,
    )

Line flow constraints

In [ ]:
    c7 = ExaModels.constraint(
        w,
        p[b.f_idx]^2 + q[b.f_idx]^2 - b.rate_a_sq for b in data.branch;
        lcon = fill!(similar(data.branch, Float64, length(data.branch)), -Inf),
    )
    c8 = ExaModels.constraint(
        w,
        p[b.t_idx]^2 + q[b.t_idx]^2 - b.rate_a_sq for b in data.branch;
        lcon = fill!(similar(data.branch, Float64, length(data.branch)), -Inf),
    )

Active power balance

In [ ]:
    c9 = ExaModels.constraint(w, b.pd + b.gs * vm[b.i]^2 for b in data.bus)
    c11 = ExaModels.constraint!(w, c9, a.bus => p[a.i] for a in data.arc)
    c13 = ExaModels.constraint!(w, c9, g.bus => -pg[g.i] for g in data.gen)

Reactive power balance

In [ ]:
    c10 = ExaModels.constraint(w, b.qd - b.bs * vm[b.i]^2 for b in data.bus)
    c12 = ExaModels.constraint!(w, c10, a.bus => q[a.i] for a in data.arc)
    c14 = ExaModels.constraint!(w, c10, g.bus => -qg[g.i] for g in data.gen)

    return ExaModels.ExaModel(w; kwargs...)
end

Before diving into the solution on the GPU, we show how to solve `case9`
using MadNLP:

In [ ]:
using MadNLP

nlp = acopf_model(data)
results = madnlp(nlp)

## Solving optimal power flow on the GPU
For solving the optimal power flow model on the GPU, the set-up is similar to
what we have detailed in the tutorial 3. We start by importing MadNLPGPU, and we
instantiate a new optimal power flow instance on the GPU:

In [ ]:
using MadNLPGPU

nlp_gpu = acopf_model(data; backend=CUDABackend())

Solving the problem using cuDSS simply amounts to

In [ ]:
results = madnlp(nlp_gpu)

The instance `case9` is too small to get any significant speed-up compared
to the CPU. However, we can solve a larger instance just by importing new data:
For instance, to solve the case `10000_goc`:

In [ ]:
data = JLD2.load("instances/pglib_opf_case10000_goc.jld2")["data"]
nlp_gpu = acopf_model(data; backend=CUDABackend())
results = madnlp(nlp_gpu; cudss_algorithm=MadNLP.LDL, max_iter=100)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*